In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import pickle

In [12]:
all_songs = pd.read_csv('all_songs.csv')

In [13]:
all_songs

,song title,lyrics,artist
0,In Da Club,"[Intro]\nGo, go, go, go, go, go\nGo Shorty, it...",50 Cent
1,21 Questions,[Ad-Libs]\nNew York City\nYou are now rockin'\...,50 Cent
2,Many Men (Wish Death),"[Skit]\nMan, we gotta go get somethin' to eat\...",50 Cent
3,My Life,"[Chorus]\nMy, yeah, yeah, mmm\nMy life, my lif...",50 Cent
4,Patiently Waiting,"[Intro]\nHey Em, you know you're my favorite w...",50 Cent
...,...,...,...
620,You Made Me Feel Love,"Well, well, well, well, well, well, well\n\nI ...",Smokey Robinson
621,The Bottom Line,Bought a pound of writing paper\nIt's balled u...,Smokey Robinson
622,Let Me Be the Clock,"Ooh, hoo, ooh\nOoh, ooh\n\nLet me be the clock...",Smokey Robinson
623,Do Like I Do,"You got to do like I do, oh, like I do\nWhen w...",Smokey Robinson


In [14]:
some_songs = all_songs.loc[all_songs['lyrics'].str.startswith('[').fillna(False)]

In [15]:
some_songs

,song title,lyrics,artist
0,In Da Club,"[Intro]\nGo, go, go, go, go, go\nGo Shorty, it...",50 Cent
1,21 Questions,[Ad-Libs]\nNew York City\nYou are now rockin'\...,50 Cent
2,Many Men (Wish Death),"[Skit]\nMan, we gotta go get somethin' to eat\...",50 Cent
3,My Life,"[Chorus]\nMy, yeah, yeah, mmm\nMy life, my lif...",50 Cent
4,Patiently Waiting,"[Intro]\nHey Em, you know you're my favorite w...",50 Cent
...,...,...,...
575,Cruisin’,"[Intro]\n(Crusin')\n\n[Verse 1]\nBaby, let's c...",Smokey Robinson
576,Really Gonna Miss You,[Verse 1]\nReally gonna miss you\nIt's really ...,Smokey Robinson
579,The Agony and the Ecstasy,[Verse 1]\nWhat's it all about this crazy love...,Smokey Robinson
582,Ooh Baby Baby,[Verse 1]\nI did you wrong\nMy heart went out ...,Smokey Robinson


In [16]:
X = some_songs.lyrics.values[:]
y = some_songs.artist.values
Y = pd.get_dummies(y).to_numpy()

In [17]:
Y = Y[:33]

In [18]:
X = X[:33]

In [19]:
from tqdm import tqdm

In [20]:
Y = torch.from_numpy(Y)

In [21]:
Y = torch.unsqueeze(Y.squeeze(), 1)
X = torch.unsqueeze(X.squeeze(), 1)

TypeError: unsqueeze(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [22]:
for ind in tqdm(range(len(X))):
    verses = split_by_verse(X[ind])
    X[ind] = get_embeddings(verses)

  0%|          | 0/33 [00:00<?, ?it/s]


NameError: name 'split_by_verse' is not defined

In [59]:
from sklearn.model_selection import train_test_split

In [207]:
Y = Y.view((33, 1, 13))
X = X.view((8, 1, 33))

TypeError: Tuple must have size 2, but has size 3

In [204]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.9, shuffle = True)

ValueError: Found input variables with inconsistent numbers of samples: [33, 1]

In [14]:
import regex as re

In [25]:
def split_by_verse(song):
    verses = song.split('[')
    for ind in range(1, len(verses)):
        verses[ind] = verses[ind].split(']')[1]
        verses[ind] = verses[ind][1:]
        verses[ind] = re.sub('\n', ' ', verses[ind])
        verses[ind] = re.sub('  ', '', verses[ind])
    return verses[1:]

In [29]:
q21 = split_by_verse(t1)

In [152]:
def get_embeddings(msgs):
    encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')
    embeddings = encoder.encode(msgs)
    return torch.unsqueeze(torch.from_numpy(embeddings), 0)

In [28]:
from sentence_transformers import SentenceTransformer

In [30]:
embs = get_embeddings(q21)

Calculating 768d embeddings...
Done


In [32]:
embs.shape

(8, 768)

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [40]:
embs_torch = torch.from_numpy(embs)

In [45]:
embs_torch = torch.unsqueeze(embs_torch, 1)

In [200]:
class BaseLSTM(nn.Module):

    def __init__(self, hidden_dim, n_artists, embedding_dim=768):
        super(BaseLSTM, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 1, batch_first= True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2art = nn.Linear(hidden_dim, n_artists)
        
        self.hid = torch.zeros(1, 1, hidden_dim)
        self.cell = torch.zeros(1,1, hidden_dim)

    def forward(self, embs):
        h0, c0 = self.hid, self.cell
        lstm_out, (h_out, c_out) = self.lstm(embs, (h0, c0))
        tag_space = self.hidden2art(lstm_out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [201]:
# training
model = BaseLSTM(256, 13)
loss_f = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-2)


    
for e in range(300):
    for x, y in zip(x_train, y_train):
        model.zero_grad()
        print(x)
        artist_scores = model(x)
        loss = loss_f(artist_scores, y)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    inputs = x_train
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[[-0.7582, -0.5377,  0.5816,  ..., -0.7437, -0.1597, -0.6536],
         [-0.4017,  0.1243,  0.3215,  ..., -0.1359,  0.0877,  0.1471],
         [-0.4822,  0.2756,  0.6754,  ..., -0.0997, -0.1210, -0.1149],
         ...,
         [-0.4408, -0.5897,  0.5616,  ..., -0.3774, -0.3277, -0.5291],
         [-0.3866,  0.0849,  0.3226,  ..., -0.1309,  0.0355,  0.1701],
         [-0.3866,  0.0849,  0.3226,  ..., -0.1309,  0.0355,  0.1701]]])


ValueError: Expected input batch_size (1) to match target batch_size (13).

In [136]:
torch.zeros([2,4]).size()[1]

4

In [147]:
Y.size()

torch.Size([33, 13])

In [ ]:
Y 